In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
from umap import UMAP

# --- 1. PREPROCESSING SETUP ---
print("Setting up cleaning tools...")
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
custom_stops = {'page', 'http', 'https', 'www', 'com', 'paragraph', 'table', 'figure', 'section'} 
stop_words.update(custom_stops)

def clean_text(text):
    if not isinstance(text, str): return ""
    # Lowercase & remove non-text characters
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize, remove stopwords, lemmatize
    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words and len(w) > 2]
    return " ".join(words)

# --- 2. LOAD DATA ---
csv_path = "output.csv"
print(f"Loading {csv_path}...")
df = pd.read_csv(csv_path)
df = df.dropna(subset=['text'])

# Apply cleaning
df['clean_text'] = df['text'].apply(clean_text)
df = df[df['clean_text'].str.len() > 5] # Remove empty results
docs = df['clean_text'].tolist()

print(f"Processing {len(docs)} documents...")

# --- 3. CONFIGURE "HYPER-GRANULAR" SETTINGS ---

# UMAP: Local Focus
# n_neighbors=2 forces the model to only look at very immediate similarities.
# This prevents it from grouping vaguely similar ideas together.
umap_model = UMAP(
    n_neighbors=2,       # EXTREME GRANULARITY (Default is 15)
    n_components=5, 
    min_dist=0.0, 
    metric='cosine'
)

# HDBSCAN: Tiny Clusters
# min_cluster_size=3 means a topic can be created with just 3 paragraphs.
# min_samples=1 ensures almost NO data is treated as noise/outliers.
hdbscan_model = HDBSCAN(
    min_cluster_size=3,  # EXTREME GRANULARITY (Default is 10)
    min_samples=1,       # PREVENTS OUTLIERS (Default is usually larger)
    metric='euclidean', 
    cluster_selection_method='eom', 
    prediction_data=True
)

# Vectorizer: Allow rare words to define topics
vectorizer_model = CountVectorizer(stop_words="english", min_df=2)

# --- 4. RUN MODEL ---
print("Fitting Hyper-Granular Model (this may generate hundreds of topics)...")

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    language="english",
    verbose=True,
    nr_topics=None # CRITICAL: Do not merge topics
)

topics, probs = topic_model.fit_transform(docs)

# --- 5. REDUCE REMAINING OUTLIERS ---
# With min_samples=1, outliers should be low, but this forces 100% assignment
try:
    new_topics = topic_model.reduce_outliers(docs, topics)
    topic_model.update_topics(docs, topics=new_topics)
    print("Outliers reduced successfully.")
except Exception as e:
    print("Skipping outlier reduction (likely almost 0 outliers exists).")

# --- 6. EXPORT RESULTS ---
freq = topic_model.get_topic_info()
print(f"\nTotal Topics Found: {len(freq) - 1}")

# Export to CSV
freq.to_csv("hyper_granular_topics.csv", index=False)
print("Saved to 'hyper_granular_topics.csv'")

# Visualize top 50 (since you will have many)
fig = topic_model.visualize_barchart(top_n_topics=50)
fig.write_html("hyper_granular_chart.html")
print("Visualization saved.")

Setting up cleaning tools...
Loading output.csv...


2026-01-20 11:37:09,785 - BERTopic - Embedding - Transforming documents to embeddings.


Processing 528 documents...
Fitting Hyper-Granular Model (this may generate hundreds of topics)...


Batches:   0%|          | 0/17 [00:00<?, ?it/s]

2026-01-20 11:37:22,456 - BERTopic - Embedding - Completed ✓
2026-01-20 11:37:22,458 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-20 11:37:35,415 - BERTopic - Dimensionality - Completed ✓
2026-01-20 11:37:35,417 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-20 11:37:35,513 - BERTopic - Cluster - Completed ✓
2026-01-20 11:37:35,515 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-20 11:37:35,641 - BERTopic - Representation - Completed ✓
100%|██████████| 1/1 [00:00<00:00, 82.56it/s]
2026-01-20 11:37:35,978 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


Outliers reduced successfully.

Total Topics Found: 108
Saved to 'hyper_granular_topics.csv'
Visualization saved.


In [2]:
# --- 5. Get All Topic Information ---

# Get the full dataframe of topic info
topic_info = topic_model.get_topic_info()

# Method A: Save to CSV (Recommended for viewing all topics easily)
topic_info.to_csv("all_topics_list.csv", index=False)
print(f"Success! Exported {len(topic_info)} topics to 'all_topics_list.csv'.")

# Method B: Print all topics to the console
# We change pandas settings to ensure rows aren't hidden
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

print("\n--- ALL TOPICS ---")
print(topic_info[['Topic', 'Count', 'Name']])

# Reset pandas options (optional)
pd.reset_option('display.max_rows')

Success! Exported 109 topics to 'all_topics_list.csv'.

--- ALL TOPICS ---
     Topic  Count  \
0        0     10   
1        1     10   
2        2      9   
3        3      9   
4        4      8   
5        5      8   
6        6      9   
7        7      8   
8        8      8   
9        9      7   
10      10      7   
11      11      7   
12      12      7   
13      13      8   
14      14      6   
15      15      6   
16      16      6   
17      17      6   
18      18      6   
19      19      6   
20      20      6   
21      21      6   
22      22      6   
23      23      6   
24      24      6   
25      25      6   
26      26      5   
27      27      5   
28      28      5   
29      29      5   
30      30      5   
31      31      5   
32      32      5   
33      33      5   
34      34      5   
35      35      5   
36      36      5   
37      37      5   
38      38      5   
39      39      7   
40      40      6   
41      41      5   
42      42      5   
4